In [ ]:
!pip install matplotlib numpy pydantic scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.model_selection import train_test_split
from pydantic import BaseModel
from typing import List
import random

# Define Type

In [ ]:
class Point(BaseModel):
    x: float
    y: float

class Line(BaseModel):
    start: Point
    end: Point

In [ ]:
class Chart(BaseModel):
    points: List[Point] = []
    lines: List[Line] = []

    def add_point(self, point: Point) -> None:
        self.points.append(point)

    def add_line(self, start: Point, end: Point) -> None:
        line = Line(start=start, end=end)
        self.lines.append(line)

    def add_points(self, points: List[Point]) -> None:
        self.points.extend(points)

    def add_lines(self, lines: List[Line]) -> None:
        self.lines.extend(lines)

    def add_function(self, start: Point, end: Point) -> None:
        self.draw_linear_two_point(start, end)

    def draw_linear_two_point(self, start: Point, end: Point) -> None:
        self.add_line(start, end)

    def plot(self) -> None:
        plt.figure(figsize=(10, 6))
        # Plot points
        self.plot_points(self.points, color='blue')

        # Plot lines
        self.plot_lines(self.lines)

        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('2D Chart with Points and Lines')
        plt.grid(True)
        plt.show()

    def plot_points(self, points: List[Point], color: str, edgecolors: str = 'none') -> None:
        for point in points:
            plt.scatter(point.x, point.y, color=color, s=100, edgecolors=edgecolors)
            plt.text(point.x, point.y, f'({point.x}, {point.y})')

    def plot_lines(self, lines: List[Line]) -> None:
        for line in lines:
            plt.plot([line.start.x, line.end.x], [line.start.y, line.end.y], color='red')
            plt.text((line.start.x + line.end.x) / 2, (line.start.y + line.end.y) / 2, 'Line', color='red')

In [ ]:
class PointFactory(BaseModel):
    chart: Chart

    def create_random_points(self, n=10, min=0, max=10) -> List[Point]:
        points = []
        for _ in range(n):
            # random float between min and max
            point = Point(x=random.uniform(min, max), y=random.uniform(min, max))
            points.append(point)
        return points

    def add_random_points(self, n=10, min=0, max=10) -> None:
        points = self.create_random_points(n, min, max)
        self.chart.add_points(points)

    def add_random_functions(self, n=10, min=0, max=10) -> None:
        previous_point = Point(x=0, y=0)
        for i in range(n):
            start = previous_point
            end = Point(x=i+1, y=i + random.uniform(min, max))
            previous_point = end
            self.chart.add_function(start, end)

In [ ]:
class SimpleSVM: # https://medium.com/@gallettilance/support-vector-machines-16241417ee6d
    def __init__(self, epochs=100, lr=0.05, expanding_rate=0.99, retracting_rate=1.01):
        self.epochs = epochs # Number of iterations
        self.lr = lr # Learning rate
        self.expanding_rate = expanding_rate # Rate at which the street expands
        self.retracting_rate = retracting_rate # Rate at which points are retracted
        self.w = None # Weights
        self.b = None # Bias
        self.history = [] # History of weights and bias

    def fit(self, X, Y):
        # Initialize weights and bias to zero
        self.w = np.zeros(X.shape[1]) # Weights
        self.b = 0.0 # Bias

        for _ in range(self.epochs):
            for i in range(len(X)):
                x, y = X[i], Y[i]

                # Calculate prediction
                ypred = np.dot(self.w, x) + self.b

                if y * ypred < 1:
                    # Misclassified or within margin
                    # learning rate * (y * x - 2 * (1/epochs) * w)
                    self.w += self.lr * (y * x - 2 * (1/self.epochs) * self.w)
                    # learning rate * y
                    self.b += self.lr * y
                else:
                    # Correctly classified
                    self.w += self.lr * -2 * (1/self.epochs) * self.w

            # Save history for debugging
            self.history.append((self.w.copy(), self.b))

    def predict(self, X):
        # Return the sign of the dot product of the weights and the input plus the bias
        return np.sign(np.dot(X, self.w) + self.b)

    def plot_decision_boundary(self, X, Y):
        plt.scatter(X[:, 0], X[:, 1], c=Y, cmap='coolwarm', s=30, edgecolors='k')

        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))

        Z = self.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')

        # Plot the decision boundary
        decision_boundary = -self.w[0] / self.w[1] * xx - self.b / self.w[1]
        plt.plot(xx[0], decision_boundary[0], 'k-', linewidth=2)

        # Plot the margin lines
        margin = 1 / np.sqrt(np.sum(self.w ** 2))
        margin_up = decision_boundary[0] + margin
        margin_down = decision_boundary[0] - margin
        plt.plot(xx[0], margin_up, 'k--', linewidth=1)
        plt.plot(xx[0], margin_down, 'k--', linewidth=1)

        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.title('SVM Decision Boundary with Margins')
        plt.show()

    def accuracy(self, X, Y):
        predictions = self.predict(X)
        return np.mean(predictions == Y)


In [ ]:
# Create an instance of the Chart class
chart = Chart()

# Create an instance of the PointFactory class
point_factory = PointFactory(chart=chart)

# Generate two groups of random points
group1 = point_factory.create_random_points(n=100, min=0, max=5)
group2 = point_factory.create_random_points(n=100, min=5, max=10)

# Add the points to the chart
chart.add_points(group1)
chart.add_points(group2)

# Plot the chart
chart.plot()

In [ ]:
# Combine the points into a single dataset
X = np.array([[point.x, point.y] for point in group1 + group2])
Y = np.array([1] * len(group1) + [-1] * len(group2))

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create an instance of the SimpleSVM class with updated parameters
svm = SimpleSVM(epochs=1000, lr=0.05, expanding_rate=0.99, retracting_rate=1.01)

# Train the SVM model
svm.fit(X_train, Y_train)

# Plot the decision boundary
svm.plot_decision_boundary(X_train, Y_train)

# Evaluate and print the accuracy
accuracy = svm.accuracy(X_test, Y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Animate the decision boundary
svm.animate(X_train, Y_train)